# Initialization

In [1]:
import sys

In [2]:
from collections import defaultdict

In [3]:
import numpy as np
import pandas as pd
import scipy

In [4]:
from sklearn.metrics import mean_squared_error, mean_absolute_error, ndcg_score
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.preprocessing import LabelEncoder, normalize

import surprise

from implicit.als import AlternatingLeastSquares

from catboost import CatBoostClassifier, Pool

/home/mle-user/mle_projects/practicum_mle_4_recsys/.env_recsys_start/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [5]:
%matplotlib inline
%config InlineBackend.figure_format = 'png'
%config InlineBackend.figure_format = 'retina'

In [6]:
def mdir(obj):
    print(*[i for i in dir(obj) if not i.startswith("_")], sep="\n")

# Загрузка данных

In [7]:
items = pd.read_parquet("items.par")
events = pd.read_parquet("events.par")

# Разбиение с учётом хронологии

Рекомендательные системы на практике работают с учётом хронологии. Поэтому поток событий для тренировки и валидации полезно делить на то, что уже случилось, и что ещё случится. Это позволяет проводить валидацию на тех же пользователях, на которых тренировались, но на их событиях в будущем.

В качестве точки разбиения используем `2017-08-01`, то есть отнесем в тестовую часть три последних месяца.

In [8]:
# зададим точку разбиения
train_test_global_time_split_date = pd.to_datetime("2017-08-01").date()

train_test_global_time_split_idx = events["started_at"] < train_test_global_time_split_date
events_train = events[train_test_global_time_split_idx]
events_test = events[~train_test_global_time_split_idx]

# количество пользователей в train и test
users_train = events_train["user_id"].drop_duplicates()
users_test = events_test["user_id"].drop_duplicates()
# количество пользователей, которые есть и в train, и в test
common_users = np.intersect1d(users_train, users_test)

print(len(users_train), len(users_test), len(common_users)) 

428220 123223 120858


---

# === Знакомство: "холодный" старт

## "Холодные" пользователи

«Холодные» пользователи — те, которые есть в `test`, но отсутствуют в `train`. Это соответствует хронологическому порядку, в котором и работает рекомендательная система.

In [9]:
cold_users = np.setdiff1d(users_test, users_train)

print(len(cold_users)) 

2365


## Топ-100 наиболее популярных книг

Теперь можно построить сами рекомендации. Для этого найдем топ-100 наиболее популярных книг (учитывая и их среднюю оценку) за последние несколько лет. Например, с 2015 года и со средней оценкой не меньше 4. Их мы будем рекомендовать «холодным» пользователям.

In [10]:
top_pop_start_date = pd.to_datetime("2015-01-01").date()

item_popularity = events_train \
    .query("started_at >= @top_pop_start_date") \
    .groupby(["item_id"]).agg(users=("user_id", "nunique"), avg_rating=("rating", "mean")).reset_index()

In [11]:
item_popularity.head()

,item_id,users,avg_rating
0,1,8728,4.796059
1,2,9528,4.685873
2,3,15139,4.706057
3,5,11890,4.770143
4,6,10679,4.764772


In [12]:
item_popularity["popularity_weighted"] = item_popularity["users"] * item_popularity["avg_rating"]

In [13]:
# сортируем по убыванию взвешенной популярности
item_popularity = item_popularity.sort_values("popularity_weighted", ascending=False)

# выбираем первые 100 айтемов со средней оценкой avg_rating не меньше 4
top_k_pop_items = item_popularity[item_popularity["avg_rating"] > 4].head(100)

In [14]:
top_k_pop_items

,item_id,users,avg_rating,popularity_weighted
32387,18007564,20207,4.321275,87320.0
32623,18143977,19462,4.290669,83505.0
30695,16096824,16770,4.301014,72128.0
2,3,15139,4.706057,71245.0
3718,38447,14611,4.232770,61845.0
...,...,...,...,...
19596,2767052,4361,4.413437,19247.0
32835,18293427,4674,4.092640,19129.0
378,3636,4667,4.098564,19128.0
33611,18966819,4361,4.374914,19079.0


Добавив информацию о книгах, можно просмотреть, какие попали в топ.

In [15]:
# добавляем информацию о книгах

top_k_pop_items = top_k_pop_items.merge(
    items.set_index("item_id")[["author", "title", "genre_and_votes", "publication_year"]], on="item_id")

with pd.option_context('display.max_rows', 100):
    display(top_k_pop_items[["item_id", "author", "title", "publication_year", "users", "avg_rating", "popularity_weighted", "genre_and_votes"]]) 


,item_id,author,title,publication_year,users,avg_rating,popularity_weighted,genre_and_votes
0,18007564,Andy Weir,The Martian,2014,20207,4.321275,87320.0,"{'Science Fiction': 11966, 'Fiction': 8430}"
1,18143977,Anthony Doerr,All the Light We Cannot See,2014,19462,4.290669,83505.0,"{'Historical-Historical Fiction': 13679, 'Fict..."
2,16096824,Sarah J. Maas,A Court of Thorns and Roses (A Court of Thorns...,2015,16770,4.301014,72128.0,"{'Fantasy': 14326, 'Young Adult': 4662, 'Roman..."
3,3,"J.K. Rowling, Mary GrandPré",Harry Potter and the Sorcerer's Stone (Harry P...,1997,15139,4.706057,71245.0,"{'Fantasy': 59818, 'Fiction': 17918, 'Young Ad..."
4,38447,Margaret Atwood,The Handmaid's Tale,1998,14611,4.232770,61845.0,"{'Fiction': 15424, 'Classics': 9937, 'Science ..."
5,15881,"J.K. Rowling, Mary GrandPré",Harry Potter and the Chamber of Secrets (Harry...,1999,13043,4.632447,60421.0,"{'Fantasy': 50130, 'Young Adult': 15202, 'Fict..."
6,11235712,Marissa Meyer,"Cinder (The Lunar Chronicles, #1)",2012,14348,4.179189,59963.0,"{'Young Adult': 10539, 'Fantasy': 9237, 'Scien..."
7,17927395,Sarah J. Maas,A Court of Mist and Fury (A Court of Thorns an...,2016,12177,4.730640,57605.0,"{'Fantasy': 10186, 'Romance': 3346, 'Young Adu..."
8,18692431,"Nicola Yoon, David Yoon","Everything, Everything",2015,14121,4.071454,57493.0,"{'Young Adult': 5175, 'Romance': 3234, 'Contem..."
9,5,"J.K. Rowling, Mary GrandPré",Harry Potter and the Prisoner of Azkaban (Harr...,2004,11890,4.770143,56717.0,"{'Fantasy': 49784, 'Young Adult': 15393, 'Fict..."


## Рекомендации

Топ-рекомендации назовём рекомендациями по умолчанию (англ. default recommendations).

Уже можно оценить их качество — для этого их следует соотнести с событиями из теста, как будто бы вы показываете эти рекомендации «холодным» пользователям, зная их предпочтения.

In [16]:
# соотнесем события из теста с топ-рекомендациями

cold_users_events_with_recs = \
    events_test[events_test["user_id"].isin(cold_users)] \
    .merge(top_k_pop_items[["item_id", "avg_rating"]], on="item_id", how="left")

In [17]:
cold_users_events_with_recs.shape

(9672, 10)

In [18]:
cold_users_events_with_recs.head()

,user_id,item_id,started_at,read_at,is_read,rating,is_reviewed,started_at_month,user_id_old,avg_rating
0,1000153,29236299,2017-08-16,2017-08-20,True,5,False,2017-08-01,001ae592ce3cdb7abb6f19b9b4d19638,NaN
1,1000153,26114463,2017-08-20,2017-09-09,True,5,False,2017-08-01,001ae592ce3cdb7abb6f19b9b4d19638,NaN
2,1000153,13262783,2017-08-12,2017-08-14,True,5,False,2017-08-01,001ae592ce3cdb7abb6f19b9b4d19638,NaN
3,1000153,29923707,2017-10-08,2017-10-13,True,4,False,2017-10-01,001ae592ce3cdb7abb6f19b9b4d19638,NaN
4,1000153,18806259,2017-08-08,2017-08-12,True,3,False,2017-08-01,001ae592ce3cdb7abb6f19b9b4d19638,NaN


In [19]:
# найдем клиентов с покрытием по рекомендациям

cold_user_recs = cold_users_events_with_recs.loc[
    cold_users_events_with_recs["avg_rating"].notna(), ["user_id", "item_id", "rating", "avg_rating"]
]

In [20]:
cold_user_recs.shape

(1912, 4)

In [21]:
cold_user_recs.head()

,user_id,item_id,rating,avg_rating
13,1000504,1885,5,4.316316
17,1000712,13496,5,4.440779
23,1001508,1885,5,4.316316
25,1002222,18966819,5,4.374914
27,1002222,15839976,4,4.150180


In [22]:
# Для какой доли событий «холодных» пользователей в events_test рекомендации в top_k_pop_items совпали по книгам

cold_users_events_with_recs["avg_rating"].notna().mean()

0.19768403639371382

## Метрики

Расчитаем метрики `rmse` и `mae` для полученных рекомендаций.

In [23]:
cold_user_recs.head()

,user_id,item_id,rating,avg_rating
13,1000504,1885,5,4.316316
17,1000712,13496,5,4.440779
23,1001508,1885,5,4.316316
25,1002222,18966819,5,4.374914
27,1002222,15839976,4,4.150180


In [24]:
rmse = mean_squared_error(cold_user_recs["rating"], cold_user_recs["avg_rating"], squared=False)
mae = mean_absolute_error(cold_user_recs["rating"], cold_user_recs["avg_rating"])

print("RMSE =", round(rmse, 3))
print(" MAE =", round(mae, 3))

RMSE = 0.776
 MAE = 0.625


### Покрытие холодных пользователей рекомендациями

Полученная метрика — значение, на которое оценка рекомендации в среднем отклоняется от истинной

Это отклонение выглядит небольшим, но в данном случае стоит оценить покрытие как по пользователям, так и по объектам, ведь рекомендации, скорее всего, имеют мало пересечений с оценками пользователей. 

In [25]:
# посчитаем покрытие холодных пользователей рекомендациями

cold_users_hit_ratio = cold_users_events_with_recs.groupby("user_id").agg(hits=("avg_rating", lambda x: (x.notna()).mean()))

print(f"Доля пользователей без релевантных рекомендаций: {(cold_users_hit_ratio['hits'] == 0).mean():.2f}")
print(f"Доля пользователей с релевантными рекомендациями: {(cold_users_hit_ratio['hits'] > 0).mean():.2f}")
print(f"\nСреднее покрытие пользователей: {cold_users_hit_ratio.loc[cold_users_hit_ratio['hits'] != 0, 'hits'].mean():.2f}")

Доля пользователей без релевантных рекомендаций: 0.59
Доля пользователей с релевантными рекомендациями: 0.41

Среднее покрытие пользователей: 0.44


«Холодных» пользователей без каких-либо релевантных рекомендаций — 59%, то есть пересечение между оценёнными книгами и рекомендациями есть только у 41%, по ним же и получено значение MAE-метрики. При этом среднее покрытие — 44%. Это значит, что большая часть «холодных» пользователей не получила никаких релевантных рекомендаций, а оставшаяся часть имеет пересечения только по 44% книг. 

---

# === Знакомство: первые персональные рекомендации

Степень разреженности матрицы оценок

In [9]:
# Степень разреженности матрицы можно оценить как отношение незаполненных ячеек к общему их количеству:

sparsity = 1 - events[["user_id", "item_id"]].drop_duplicates().shape[0] / (events["user_id"].nunique() * events["item_id"].nunique())
sparsity


0.9993451160571009

In [10]:
# Степень разреженности матрицы можно оценить как отношение незаполненных ячеек к общему их количеству:

sparsity_train = 1 - events_train[["user_id", "item_id"]].drop_duplicates().shape[0] / (events_train["user_id"].nunique() * events_train["item_id"].nunique())
sparsity_train


0.9993622676592138

## Реализация SVD-алгоритма

Воспользуемся готовой реализацией SVD-алгоритма из библиотеки `surprise`. В качестве разбиения данных на `train` и `test` возьмём разбиение из предыдущего урока: `events_train`, `events_test`.

In [11]:
mdir(surprise)

AlgoBase
BaselineOnly
CoClustering
Dataset
KNNBaseline
KNNBasic
KNNWithMeans
KNNWithZScore
NMF
NormalPredictor
Prediction
PredictionImpossible
Reader
SVD
SVDpp
SlopeOne
Trainset
accuracy
builtin_datasets
dataset
dump
get_dataset_dir
get_distribution
model_selection
prediction_algorithms
reader
similarities
trainset
utils


In [12]:
# используем Reader из библиотеки surprise для преобразования событий (events)
# в формат, необходимый surprise

reader = surprise.Reader(rating_scale=(1, 5))
surprise_train_set = surprise.Dataset.load_from_df(events_train[['user_id', 'item_id', 'rating']], reader)

In [13]:
print(surprise_train_set)
print()
mdir(surprise_train_set)


build_full_trainset
construct_testset
construct_trainset
df
has_been_split
load_builtin
load_from_df
load_from_file
load_from_folds
raw_ratings
read_ratings
reader


In [14]:
surprise_train_set = surprise_train_set.build_full_trainset()

In [15]:
print(surprise_train_set)
print()
mdir(surprise_train_set)


all_items
all_ratings
all_users
build_anti_testset
build_testset
global_mean
ir
knows_item
knows_user
n_items
n_ratings
n_users
rating_scale
to_inner_iid
to_inner_uid
to_raw_iid
to_raw_uid
ur


In [16]:
# инициализируем модель

svd_model = surprise.SVD(n_factors=100, random_state=0)

In [17]:
# обучаем модель
# 2 min

svd_model.fit(surprise_train_set)

Используя обученную модель, получим рекомендации для тестовой выборки:

In [18]:
surprise_test_set = list(events_test[['user_id', 'item_id', 'rating']].itertuples(index=False))

# получаем рекомендации для тестовой выборки
svd_predictions = svd_model.test(surprise_test_set) 

In [19]:
type(svd_predictions)

list

In [20]:
len(svd_predictions)

424962

In [21]:
svd_predictions[:3]

[Prediction(uid=1000003, iid=25893709, r_ui=4, est=3.146647541664683, details={'was_impossible': False}),
 Prediction(uid=1000005, iid=34076952, r_ui=5, est=4.0248936128868875, details={'was_impossible': False}),
 Prediction(uid=1000006, iid=18774964, r_ui=4, est=4.703536489983392, details={'was_impossible': False})]

## Оценка рекомендаций

Полученные рекомендации можно оценить, используя встроенный модуль `accuracy` из библиотеки `surprise`:

In [22]:
rmse = surprise.accuracy.rmse(svd_predictions)
mae = surprise.accuracy.mae(svd_predictions)

print()
print(rmse, mae) 

RMSE: 0.8263
MAE:  0.6460

0.826346375350908 0.6460143973270805


### Проверка метрик на адекватность

Для генерации случайных рекомендаций библиотека surprise предлагает класс NormalPredictor, который выдаёт случайные рейтинги из нормального распределения.

In [23]:
# инициализируем состояние генератора, это необходимо для получения
# одной и той же последовательности случайных чисел, только в учебных целях
np.random.seed(0)

random_model = surprise.NormalPredictor()

random_model.fit(surprise_train_set)
random_predictions = random_model.test(surprise_test_set) 

In [24]:
len(random_predictions)

424962

In [25]:
random_predictions[:3]

[Prediction(uid=1000003, iid=25893709, r_ui=4, est=5, details={'was_impossible': False}),
 Prediction(uid=1000005, iid=34076952, r_ui=5, est=4.3272637894387875, details={'was_impossible': False}),
 Prediction(uid=1000006, iid=18774964, r_ui=4, est=4.8785393008980575, details={'was_impossible': False})]

In [26]:
rmse_dummy = surprise.accuracy.rmse(random_predictions)
mae_dummy = surprise.accuracy.mae(random_predictions)

print()
print(rmse_dummy, mae_dummy) 

RMSE: 1.2590
MAE:  0.9982

1.2590325375790072 0.9982060614713011


In [27]:
# На сколько процентов MAE для случайных рекомендаций от NormalPredictor выше значения MAE от SVD?

print(f"{mae_dummy / mae - 1:.2%}")

54.52%


## Рекомендации (предсказания)

In [28]:
mdir(svd_model)

bi
biased
bsl_options
bu
compute_baselines
compute_similarities
default_prediction
estimate
fit
get_neighbors
init_mean
init_std_dev
lr_bi
lr_bu
lr_pu
lr_qi
n_epochs
n_factors
predict
pu
qi
random_state
reg_bi
reg_bu
reg_pu
reg_qi
sgd
sim_options
test
trainset
verbose


In [29]:
USER_ID = 1296647

In [30]:
items_to_predict = events['item_id'].unique()

In [31]:
items_to_predict.shape

(41673,)

In [32]:
scores = [svd_model.predict(USER_ID, item_id).est for item_id in items_to_predict]

In [33]:
predictions = pd.DataFrame({
    "item_id": items_to_predict,
    "scores": scores
}).sort_values('scores', ascending=False).reset_index(drop=True)

In [34]:
predictions

,item_id,scores
0,8471387,5.000000
1,24812,5.000000
2,481749,5.000000
3,30688013,4.996900
4,1108124,4.979711
...,...,...
41668,5557442,2.360372
41669,783291,2.254230
41670,6017893,2.215569
41671,16075935,2.208172


In [35]:
def get_recommendations_svd(user_id, events, model, include_seen=True, n=5):

    """ возвращает n рекомендаций для user_id """
    
    # получим список идентификаторов всех книг
    all_items = events['item_id'].unique()
        
    # учитываем флаг, стоит ли уже прочитанные книги включать в рекомендации
    if include_seen:
        items_to_predict = all_items
    else:
        # получим список книг, которые пользователь уже прочитал ("видел")
        seen_items = events.loc[events["user_id"]==user_id, 'item_id'].unique()
        
        # книги, которые пользователь ещё не читал
        # только их и будем включать в рекомендации
        items_to_predict = np.setdiff1d(all_items, seen_items)
    
    # получаем скоры для списка книг, т. е. рекомендации
    scores = [model.predict(USER_ID, item_id).est for item_id in items_to_predict]
    
    predictions = pd.DataFrame({
        "item_id": items_to_predict,
        "scores": scores
    }).sort_values('scores', ascending=False).reset_index(drop=True).head(n)
    
    return predictions

In [36]:
get_recommendations_svd(USER_ID, events_train, svd_model, False, n=10)

,item_id,scores
0,8471387,5.000000
1,24812,5.000000
2,481749,5.000000
3,30688013,4.996900
4,1108124,4.979711
5,6898978,4.969303
6,24019187,4.964964
7,191139,4.947385
8,11221285,4.946124
9,862041,4.942287


## Дополнительная проверка качества рекомендаций

Некоторые проблемы рекомендаций отследить с помощью метрик сложно.

Например, с помощью метрик сложно выявить рекомендации с низким разнообразием — когда пользователю, который прочитал книгу одного автора, рекомендуются книги преимущественно того же автора.

Такие проблемы проще отследить «глазами». 

In [37]:
# выберем произвольного пользователя из тренировочной выборки ("прошлого")
user_id = events_train['user_id'].sample().iat[0]

print(f"user_id: {user_id}")

user_id: 1109853


In [38]:
print("История (последние события, recent)")
user_history = (
    events_train
    .query("user_id == @user_id")
    .merge(items.set_index("item_id")[["author", "title", "genre_and_votes"]], on="item_id")
)
user_history_to_print = user_history[["author", "title", "started_at", "read_at", "rating", "genre_and_votes"]].tail(10)
display(user_history_to_print)

История (последние события, recent)


,author,title,started_at,read_at,rating,genre_and_votes
0,Patrick Rothfuss,"The Wise Man's Fear (The Kingkiller Chronicle,...",2015-06-17,2015-08-31,5,"{'Fantasy': 16491, 'Fiction': 2222, 'Fantasy-E..."
1,Andy Weir,The Martian,2014-12-07,2014-12-11,4,"{'Science Fiction': 11966, 'Fiction': 8430}"
2,Brandon Sanderson,"The Way of Kings (The Stormlight Archive, #1)",2015-08-31,2015-10-30,5,"{'Fantasy': 14291, 'Fiction': 1623, 'Fantasy-E..."
3,Brandon Sanderson,"Words of Radiance (The Stormlight Archive, #2)",2015-10-30,2016-03-17,5,"{'Fantasy': 8542, 'Fiction': 872, 'Fantasy-Epi..."
4,Ken Follett,"Fall of Giants (The Century Trilogy, #1)",2016-05-12,2016-08-30,4,"{'Historical-Historical Fiction': 4665, 'Ficti..."


In [39]:
print("Рекомендации")
user_recommendations = get_recommendations_svd(user_id, events_train, svd_model, n=10)
user_recommendations = user_recommendations.merge(items[["item_id", "author", "title", "genre_and_votes"]], on="item_id")
display(user_recommendations) 

Рекомендации


,item_id,scores,author,title,genre_and_votes
0,8471387,5.000000,أمل دنقل,لا تصالح,{'Poetry': 110}
1,24812,5.000000,Bill Watterson,The Complete Calvin and Hobbes,"{'Sequential Art-Comics': 867, 'Humor': 378, '..."
2,481749,5.000000,James E. Talmage,Jesus the Christ,"{'Religion': 451, 'Christianity-Lds': 256, 'No..."
3,30688013,4.996900,Robin Hobb,"Assassin's Fate (The Fitz and the Fool, #3)","{'Fantasy': 1657, 'Fiction': 172, 'Fantasy-Epi..."
4,1108124,4.979711,Kalki,பொன்னியின் செல்வன் [Ponniyin Selvan],{'Historical-Historical Fiction': 38}
5,6898978,4.969303,"Leo Tolstoy, Rubens Figueiredo",Anna Kariênina,"{'Classics': 18053, 'Fiction': 6990, 'Cultural..."
6,24019187,4.964964,Brandon Stanton,Humans of New York: Stories,"{'Nonfiction': 662, 'Art-Photography': 190, 'A..."
7,191139,4.947385,Dr. Seuss,"Oh, The Places You'll Go!","{'Childrens': 2345, 'Childrens-Picture Books':..."
8,11221285,4.946124,Brandon Sanderson,"The Way of Kings, Part 2 (The Stormlight Archi...","{'Fantasy': 641, 'Fiction': 46, 'Fantasy-Epic ..."
9,862041,4.942287,J.K. Rowling,"Harry Potter Boxset (Harry Potter, #1-7)","{'Fantasy': 3211, 'Young Adult': 1068, 'Fictio..."


---

# === Базовые подходы: коллаборативная фильтрация

Получим рекомендации с помощью алгоритма ALS из библиотеки `implicit`. Схема действий такая:
1. Перекодировать идентификаторы пользователей и объектов.
1. Явно подготовить матрицу взаимодействий.
1. Провести тренировку модели с помощью ALS на тренировочной части данных.
1. На основе полученной модели сгенерировать рекомендации.
1. Оценить полученные рекомендации.

## Перекодировать идентификаторы пользователей и объектов.

In [9]:
# перекодируем идентификаторы пользователей: 
# из имеющихся в последовательность 0, 1, 2, ...

user_encoder = LabelEncoder()
user_encoder.fit(events["user_id"])

LabelEncoder()

In [10]:
events_train["user_id_enc"] = user_encoder.transform(events_train["user_id"])
events_test["user_id_enc"] = user_encoder.transform(events_test["user_id"])

/tmp/ipykernel_17007/4073530557.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  events_train["user_id_enc"] = user_encoder.transform(events_train["user_id"])
/tmp/ipykernel_17007/4073530557.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  events_test["user_id_enc"] = user_encoder.transform(events_test["user_id"])


In [11]:
# перекодируем идентификаторы объектов: 
# из имеющихся в последовательность 0, 1, 2, ...

item_encoder = LabelEncoder()
item_encoder.fit(items["item_id"])

LabelEncoder()

In [12]:
items["item_id_enc"] = item_encoder.transform(items["item_id"])
events_train = events_train.merge(items[["item_id", "item_id_enc"]], on="item_id", how="left")
events_test = events_test.merge(items[["item_id", "item_id_enc"]], on="item_id", how="left")

In [13]:
# Какое максимальное значение получилось для events_train[’item_id_enc']?

events_train['item_id_enc'].max()

43304

## Подготовить матрицу взаимодействий.

In [14]:
# Оценим размер полной матрицы

matrix_size = events_train["user_id"].nunique() * events_train["item_id"].nunique()
matrix_size_gb = matrix_size / (1024 ** 3)

matrix_size_gb

16.54028546065092

In [15]:
# создаём sparse-матрицу формата CSR

user_item_matrix_train = scipy.sparse.csr_matrix(
    (
        events_train["rating"],
        (events_train['user_id_enc'], events_train['item_id_enc'])
    ),
    dtype=np.int8
) 

In [16]:
# Размер разряженной матрицы

sum([sys.getsizeof(i) for i in user_item_matrix_train.data])/1024**3 

0.26370687410235405

## Провести тренировку модели с помощью ALS на тренировочной части данных.

In [17]:
# Создадим и обучим модель
# 3 min

als_model = AlternatingLeastSquares(factors=50, iterations=50, regularization=0.05, random_state=0)

als_model.fit(user_item_matrix_train) 

/home/mle-user/mle_projects/practicum_mle_4_recsys/.env_recsys_start/lib/python3.10/site-packages/implicit/cpu/als.py:95: RuntimeWarning: OpenBLAS is configured to use 4 threads. It is highly recommended to disable its internal threadpool by setting the environment variable 'OPENBLAS_NUM_THREADS=1' or by calling 'threadpoolctl.threadpool_limits(1, "blas")'. Having OpenBLAS use a threadpool can lead to severe performance issues here.
  check_blas_config()
100%|██████████| 50/50 [03:07<00:00,  3.75s/it]


## На основе полученной модели сгенерировать рекомендации.

In [18]:
def get_recommendations_als(user_item_matrix, model, user_id, user_encoder, item_encoder, include_seen=True, n=5):
    """
    Возвращает отранжированные рекомендации для заданного пользователя
    """
    user_id_enc = user_encoder.transform([user_id])[0]
    recommendations = model.recommend(
         user_id_enc, 
         user_item_matrix[user_id_enc], 
         filter_already_liked_items=not include_seen,
         N=n)
    recommendations = pd.DataFrame({"item_id_enc": recommendations[0], "score": recommendations[1]})
    recommendations["item_id"] = item_encoder.inverse_transform(recommendations["item_id_enc"])
    
    return recommendations 

In [19]:
get_recommendations_als(
    user_item_matrix_train,
    als_model,
    1430584,
    user_encoder,
    item_encoder,
    include_seen=True,
    n=5
)

,item_id_enc,score,item_id
0,41809,0.472945,29056083
1,34434,0.463936,17167166
2,35669,0.460429,17927395
3,33783,0.454709,16096824
4,37759,0.445634,20613470


### Дополнительная проверка качества рекомендаций

Некоторые проблемы рекомендаций отследить с помощью метрик сложно.

Например, с помощью метрик сложно выявить рекомендации с низким разнообразием — когда пользователю, который прочитал книгу одного автора, рекомендуются книги преимущественно того же автора.

Такие проблемы проще отследить «глазами». 

In [20]:
# выберем произвольного пользователя из тренировочной выборки ("прошлого")
user_id = events_train['user_id'].sample().iat[0]

print(f"user_id: {user_id}")

user_id: 1185580


In [21]:
print("История (последние события, recent)")
user_history = (
    events_train
    .query("user_id == @user_id")
    .merge(items.set_index("item_id")[["author", "title", "genre_and_votes"]], on="item_id")
)
user_history_to_print = user_history[["author", "title", "started_at", "read_at", "rating", "genre_and_votes"]].tail(10)
display(user_history_to_print)

История (последние события, recent)


,author,title,started_at,read_at,rating,genre_and_votes
65,Jeffrey Eugenides,The Marriage Plot,2012-09-04,2012-09-04,3,"{'Fiction': 2339, 'Contemporary': 326, 'Romanc..."
66,David Wong,"John Dies at the End (John Dies at the End, #1)",2012-12-25,2012-12-30,1,"{'Horror': 2015, 'Fiction': 1003, 'Humor': 637..."
67,Jenny Lawson,Let's Pretend This Never Happened: A Mostly Tr...,2012-05-26,2012-06-13,5,"{'Nonfiction': 2988, 'Autobiography-Memoir': 2..."
68,Daniel O'Malley,"The Rook (The Checquy Files, #1)",2014-01-29,2014-02-09,5,"{'Fantasy': 2731, 'Fantasy-Urban Fantasy': 108..."
69,David Grann,The Lost City of Z: A Tale of Deadly Obsession...,2012-07-23,2012-07-29,4,"{'Nonfiction': 2844, 'History': 1643, 'Adventu..."
70,"Lene Kaaberbøl, Agnete Friis","The Boy in the Suitcase (Nina Borg, #1)",2012-05-29,2012-06-01,4,"{'Mystery': 638, 'Fiction': 406, 'Thriller': 2..."
71,Tom Rachman,The Imperfectionists,2011-04-29,2011-04-30,3,"{'Fiction': 1301, 'Short Stories': 156, 'Cultu..."
72,James Frey,Bright Shiny Morning,2012-06-21,2012-06-26,4,"{'Fiction': 282, 'Contemporary': 24, 'Novels':..."
73,Colum McCann,Let the Great World Spin,2011-07-25,2011-07-26,4,"{'Fiction': 2046, 'Historical-Historical Ficti..."
74,Joe Hill,NOS4A2,2013-08-02,2013-08-12,4,"{'Horror': 4763, 'Fiction': 1138, 'Fantasy': 8..."


In [22]:
print("Рекомендации")
user_recommendations = get_recommendations_als(
    user_item_matrix_train,
    als_model,
    user_id,
    user_encoder,
    item_encoder,
    include_seen=True,
    n=10,
)
user_recommendations = user_recommendations.merge(items[["item_id", "author", "title", "genre_and_votes"]], on="item_id")
display(user_recommendations) 

Рекомендации


,item_id_enc,score,item_id,author,title,genre_and_votes
0,20957,0.971113,2767052,Suzanne Collins,"The Hunger Games (The Hunger Games, #1)","{'Young Adult': 30042, 'Fiction': 16754, 'Scie..."
1,23287,0.953350,6148028,Suzanne Collins,"Catching Fire (The Hunger Games, #2)","{'Young Adult': 25542, 'Science Fiction-Dystop..."
2,25505,0.937294,7260188,Suzanne Collins,"Mockingjay (The Hunger Games, #3)","{'Young Adult': 24130, 'Science Fiction-Dystop..."
3,33021,0.825513,15783514,Neil Gaiman,The Ocean at the End of the Lane,"{'Fantasy': 15115, 'Fiction': 6563, 'Horror': ..."
4,27664,0.777785,9460487,Ransom Riggs,Miss Peregrine’s Home for Peculiar Children (M...,"{'Fantasy': 12454, 'Young Adult': 9293, 'Ficti..."
5,27575,0.720089,9361589,Erin Morgenstern,The Night Circus,"{'Fantasy': 16749, 'Fiction': 7058, 'Romance':..."
6,30236,0.647196,12262741,Cheryl Strayed,Wild: From Lost to Found on the Pacific Crest ...,"{'Nonfiction': 6335, 'Autobiography-Memoir': 4..."
7,1641,0.618131,13496,George R.R. Martin,"A Game of Thrones (A Song of Ice and Fire, #1)","{'Fantasy': 44086, 'Fiction': 10111, 'Fantasy-..."
8,27634,0.591377,9418327,Tina Fey,Bossypants,"{'Nonfiction': 8118, 'Autobiography-Memoir': 4..."
9,1232,0.589340,10572,George R.R. Martin,"A Clash of Kings (A Song of Ice and Fire, #2)","{'Fantasy': 31452, 'Fiction': 6543, 'Fantasy-E..."


### Рекомендации для всех пользователей

In [23]:
# получаем список всех возможных user_id (перекодированных)

user_ids_enc = user_encoder.transform(user_encoder.classes_)

In [24]:
# получаем рекомендации для всех пользователей
# 2 min

item_ids_enc, als_scores = als_model.recommend(
    user_ids_enc, 
    user_item_matrix_train[user_ids_enc], 
    filter_already_liked_items=False,
    N=100
) 

In [25]:
item_ids_enc.shape

(430585, 100)

In [26]:
item_ids_enc

array([[    2,  1942,     3, ..., 28836, 30688, 10393],
       [31432, 29792, 36956, ...,   533, 32060, 34554],
       [35810, 33276, 37255, ..., 31562, 41459,  1043],
       ...,
       [20997, 20386, 23004, ...,  2293, 28200, 29560],
       [22844, 28025, 37138, ..., 37914,   422,  4112],
       [41809, 34434, 35669, ..., 33675, 28263, 22072]], dtype=int32)

In [27]:
# преобразуем полученные рекомендации в табличный формат

als_recommendations_df = pd.DataFrame({
    "user_id_enc": user_ids_enc,
    "item_id_enc": item_ids_enc.tolist(), 
    "score": als_scores.tolist()})

als_recommendations_df = als_recommendations_df.explode(["item_id_enc", "score"], ignore_index=True)

In [28]:
als_recommendations_df.shape

(43058500, 3)

In [29]:
als_recommendations_df.head(10)

,user_id_enc,item_id_enc,score
0,0,2,0.990941
1,0,1942,0.896617
2,0,3,0.864404
3,0,4,0.822254
4,0,1,0.774095
5,0,0,0.735119
6,0,8250,0.720993
7,0,1641,0.685299
8,0,493,0.593034
9,0,39686,0.570642


In [30]:
als_recommendations_df.dtypes

user_id_enc     int64
item_id_enc    object
score          object
dtype: object

In [31]:
# приводим типы данных

als_recommendations_df["item_id_enc"] = als_recommendations_df["item_id_enc"].astype("int")
als_recommendations_df["score"] = als_recommendations_df["score"].astype("float")

# получаем изначальные идентификаторы

als_recommendations_df["user_id"] = user_encoder.inverse_transform(als_recommendations_df["user_id_enc"])
als_recommendations_df["item_id"] = item_encoder.inverse_transform(als_recommendations_df["item_id_enc"])

als_recommendations_df = als_recommendations_df.drop(columns=["user_id_enc", "item_id_enc"])[["user_id", "item_id", "score"]]

In [32]:
als_recommendations_df.shape

(43058500, 3)

In [33]:
als_recommendations_df.head()

,user_id,item_id,score
0,1000000,3,0.990941
1,1000000,15881,0.896617
2,1000000,5,0.864404
3,1000000,6,0.822254
4,1000000,2,0.774095


### Сохраним полученные рекомендации в файл, они ещё нам пригодятся.

In [34]:
als_recommendations_df.to_parquet("als_recommendations.parquet") 

## Оценить полученные рекомендации. <a id='_ngdc'></a>

In [35]:
# Для удобства оценки добавим в датафрейм с рекомендациями истинные оценки из тестовой выборки:

als_recommendations_df = (
    als_recommendations_df
    .merge(
        events_test[["user_id", "item_id", "rating"]].rename(columns={"rating": "rating_test"}), 
        on=["user_id", "item_id"],
        how="left"
    )
)

In [36]:
als_recommendations_df.head()

,user_id,item_id,score,rating_test
0,1000000,3,0.990941,NaN
1,1000000,15881,0.896617,NaN
2,1000000,5,0.864404,NaN
3,1000000,6,0.822254,NaN
4,1000000,2,0.774095,NaN


In [37]:
(
    als_recommendations_df["rating_test"].notna().sum(),
    als_recommendations_df["rating_test"].notna().sum() / als_recommendations_df.shape[0]
)

(80494, 0.0018694102209784362)

Подсчитать метрику NDCG для одного пользователя

In [38]:
def compute_ndcg(rating: pd.Series, score: pd.Series, k):

    """ подсчёт ndcg
    rating: истинные оценки
    score: оценки модели
    k: количество айтемов (по убыванию score) для оценки, остальные - отбрасываются
    """
    
    # если кол-во объектов меньше 2, то NDCG - не определена
    if len(rating) < 2:
        return np.nan

    ndcg = ndcg_score(np.asarray([rating.to_numpy()]), np.asarray([score.to_numpy()]), k=k)

    return ndcg 

In [39]:
ndcg_at_5_scores = (
    als_recommendations_df.loc[als_recommendations_df["rating_test"].notna()]
    .groupby("user_id")
    .apply(lambda x: compute_ndcg(x["rating_test"], x["score"], k=5))
) 

In [40]:
ndcg_at_5_scores.shape

(48135,)

In [41]:
ndcg_at_5_scores.head()

user_id
1000006    1.0
1000007    NaN
1000019    NaN
1000020    NaN
1000023    1.0
dtype: float64

In [42]:
# средняя метрика

ndcg_at_5_scores.mean()

0.975946709792109

In [43]:
# для какой доли пользователей удалось посчитать метрику NDCG

ndcg_at_5_scores.notna().sum() / ndcg_at_5_scores.shape[0]

0.35807624389737197

---

# === Базовые подходы: контентные рекомендации

### Перекодировать идентификаторы пользователей и объектов.

In [9]:
# перекодируем идентификаторы объектов: 
# из имеющихся в последовательность 0, 1, 2, ...

item_encoder = LabelEncoder()
item_encoder.fit(items["item_id"])

items["item_id_enc"] = item_encoder.transform(items["item_id"])

### Составим список всех жанров <a id='_get_genres_'></a>

In [10]:
items.head()

,item_id,author,title,description,genre_and_votes,num_pages,average_rating,ratings_count,text_reviews_count,publisher,publication_year,country_code,language_code,format,is_ebook,isbn,isbn13,genre_and_votes_dict,genre_and_votes_str,item_id_enc
3,6066819,Jennifer Weiner,Best Friends Forever,Addie Downs and Valerie Adler were eight when ...,"{'Womens Fiction-Chick Lit': 739, 'Fiction': 442}",368,3.49,51184,3282,Atria Books,2009,US,eng,Hardcover,False,0743294297,9780743294294,"{'Academic': None, 'Academic-Academia': None, ...","Womens Fiction-Chick Lit 739, Fiction 442",23133
6,378460,Michael Halberstam,The Wanting of Levine,,"{'Politics': 1, 'Humor': 1}",<NA>,4.38,12,4,Berkley Publishing Group,1979,US,,Paperback,False,0425040887,9780425040881,"{'Academic': None, 'Academic-Academia': None, ...","Politics 1user, Humor 1user",12687
15,89375,"Don Piper, Cecil Murphey",90 Minutes in Heaven: A True Story of Death an...,As he is driving home from a minister's confer...,"{'Christian': 395, 'Nonfiction': 392, 'Religio...",<NA>,3.91,68157,2885,,<NA>,US,,,False,0800759494,9780800759490,"{'Academic': None, 'Academic-Academia': None, ...","Christian 395, Nonfiction 392, Religion 142, S...",6460
16,89376,Randy Alcorn,Heaven,What is Heaven really going to be like? What w...,"{'Christian': 225, 'Religion-Theology': 154, '...",533,4.26,7345,566,,<NA>,US,eng,,False,0842379428,9780842379427,"{'Academic': None, 'Academic-Academia': None, ...","Christian 225, Religion-Theology 154, Nonficti...",6461
17,89377,Jennifer L. Holm,Penny from Heaven,It's 1953 and 11-year-old Penny dreams of a su...,"{'Historical-Historical Fiction': 284, 'Childr...",288,3.98,6949,615,Random House Books for Young Readers,2006,US,,Hardcover,False,037583687X,9780375836879,"{'Academic': None, 'Academic-Academia': None, ...","Historical-Historical Fiction 284, Childrens-M...",6462


In [11]:
items.dtypes

item_id                   int64
author                   object
title                    object
description              object
genre_and_votes          object
num_pages                 Int64
average_rating          float64
ratings_count             Int64
text_reviews_count        int64
publisher                object
publication_year          Int64
country_code             object
language_code            object
format                   object
is_ebook                   bool
isbn                     object
isbn13                   object
genre_and_votes_dict     object
genre_and_votes_str      object
item_id_enc               int64
dtype: object

In [12]:
items.iloc[0].to_dict()

{'item_id': 6066819,
 'author': 'Jennifer Weiner',
 'title': 'Best Friends Forever',
 'description': "Addie Downs and Valerie Adler were eight when they first met and decided to be best friends forever. But, in the wake of tragedy and betrayal during their teenage years, everything changed. Val went on to fame and fortune. Addie stayed behind in their small Midwestern town. Destiny, however, had more in store for these two. And when, twenty-five years later, Val shows up at Addie's front door with blood on her coat and terror on her face, it is the beginning of a wild adventure for two women joined by love and history who find strength together that they could not find alone.",
 'genre_and_votes': "{'Womens Fiction-Chick Lit': 739, 'Fiction': 442}",
 'num_pages': 368,
 'average_rating': 3.49,
 'ratings_count': 51184,
 'text_reviews_count': 3282,
 'publisher': 'Atria Books',
 'publication_year': 2009,
 'country_code': 'US',
 'language_code': 'eng',
 'format': 'Hardcover',
 'is_ebook': F

In [13]:
# Сначала преобразуем значения в genre_and_votes из текстового представления в тип в Python:

items["genre_and_votes"] = items["genre_and_votes"].apply(eval) 

Составим список жанров с долями голосов по ним

In [14]:
def get_genres(genre_and_votes):

    """ 
    извлекает список жанров по всем книгам, 
    подсчитывает долю голосов по каждому их них
    """
    
    genres_counter = defaultdict(int)
    
    for dict_votes in genre_and_votes.to_list():
        if dict_votes is None or not isinstance(dict_votes, dict):
            continue
        for genre, votes in dict_votes.items():
            # увеличиваем счётчик жанров
            genres_counter[genre] += int(votes)

    genres_df = (
        pd.Series(genres_counter, name="votes")
        .sort_values(ascending=False)
        .to_frame()
        .reset_index()
        .rename(columns={"index": "name"})
    )
    # genres_df.index.name = "genre_id"
    
    return genres_df

In [15]:
genres = get_genres(items["genre_and_votes"])

In [16]:
genres.shape

(815, 2)

In [17]:
genres.head()

,name,votes
0,Fantasy,6850115
1,Fiction,6406698
2,Classics,3415071
3,Young Adult,3297027
4,Romance,2422690


### Подготовим матрицы

In [18]:
def get_item2genre_matrix(genres, items):

    '''Функция строит матрицу вида «книга-жанр».
    Индексы строк не соовтетствуют исходным интексам!!! - это новые индексы !!! Поскольку важны жанры
    '''

    genre_names_to_id = genres.reset_index().set_index("name")["index"].to_dict()
    
    # list to build CSR matrix
    genres_csr_data = []
    genres_csr_row_idx = []
    genres_csr_col_idx = []
    
    for item_idx, (k, v) in enumerate(items.iterrows()):
        if v["genre_and_votes"] is None:
            continue
        for genre_name, votes in v["genre_and_votes"].items():
            genre_idx = genre_names_to_id[genre_name]
            genres_csr_data.append(int(votes))
            genres_csr_row_idx.append(item_idx)
            genres_csr_col_idx.append(genre_idx)

    genres_csr = scipy.sparse.csr_matrix(
        (genres_csr_data, (genres_csr_row_idx, genres_csr_col_idx)),
        shape=(len(items), len(genres))
    )
    
    # нормализуем, чтобы сумма оценок принадлежности к жанру была равна 1
    genres_csr = normalize(genres_csr, norm='l1', axis=1)
    
    return genres_csr 

In [19]:
items = items.sort_values(by="item_id_enc")
print(items.shape)
items.tail()

(43312, 20)


,item_id,author,title,description,genre_and_votes,num_pages,average_rating,ratings_count,text_reviews_count,publisher,publication_year,country_code,language_code,format,is_ebook,isbn,isbn13,genre_and_votes_dict,genre_and_votes_str,item_id_enc
1229539,36411879,Lylah James,The Mafia And His Angel: Part 2 (Tainted Heart...,Ayla\nThe darkness never truly left me. It's a...,"{'Romance': 38, 'Dark': 37, 'Sociology-Abuse':...",<NA>,4.64,408,87,,<NA>,US,,,True,,,"{'Academic': None, 'Academic-Academia': None, ...","Romance 38, Dark 37, Sociology-Abuse 17",43307
534649,36421066,David Anderson,The Remnant,"Will, Kevin, Rose, Quentin and Fiona have over...",{'Science Fiction Fantasy': 1},<NA>,5.00,2,1,BWL Publishing Inc.,2017,US,eng,ebook,True,,9781773628875,"{'Academic': None, 'Academic-Academia': None, ...",Science Fiction Fantasy 1user,43308
1541052,36430456,Elaine Williams Crockett,Do Not Ask,The President's beautiful twin daughters disap...,"{'Fiction': 4, 'Mystery': 3, 'Thriller': 2}",<NA>,5.00,3,3,BookBaby,2017,US,,ebook,True,,9781543910445,"{'Academic': None, 'Academic-Academia': None, ...","Fiction 4, Mystery 3, Thriller 2",43309
2251007,36515426,Logan Chance,Heartbreaker,They call me a heartbreaker.\nI've constructed...,"{'Romance': 20, 'Contemporary': 6}",<NA>,4.48,185,49,,<NA>,US,,,True,,,"{'Academic': None, 'Academic-Academia': None, ...","Romance 20, Contemporary 6",43310
96267,36524503,Emily Lloyd-Jones,The Bone Houses,A historical horror-fairy tale that follows a ...,"{'Fantasy': 841, 'Young Adult': 437, 'Horror':...",<NA>,2.00,1,1,"Little, Brown Books for Young Readers",2019,US,,,False,,,"{'Academic': None, 'Academic-Academia': None, ...","Fantasy 841, Young Adult 437, Horror 356, Hist...",43311


In [20]:
# Проверка, что декодированные индексы будут соовтетствовать новым индексам:
assert items.iloc[-1]["item_id_enc"] == items.shape[0] - 1

all_items_genres_csr = get_item2genre_matrix(genres, items)

In [21]:
all_items_genres_csr

<Compressed Sparse Row sparse matrix of dtype 'float64'
	with 210895 stored elements and shape (43312, 815)>

In [22]:
all_items_genres_csr[0].toarray()

array([[0.52498784, 0.1480262 , 0.01406379, 0.17065499, 0.        ,
        0.        , 0.        , 0.        , 0.        , 0.        ,
        0.        , 0.        , 0.        , 0.        , 0.        ,
        0.        , 0.        , 0.0305828 , 0.        , 0.        ,
        0.        , 0.        , 0.        , 0.        , 0.        ,
        0.        , 0.        , 0.        , 0.        , 0.        ,
        0.        , 0.        , 0.        , 0.        , 0.02194992,
        0.        , 0.        , 0.        , 0.04316441, 0.        ,
        0.01518392, 0.01719788, 0.        , 0.        , 0.        ,
        0.        , 0.        , 0.        , 0.        , 0.        ,
        0.        , 0.        , 0.        , 0.        , 0.        ,
        0.        , 0.        , 0.        , 0.        , 0.        ,
        0.01418825, 0.        , 0.        , 0.        , 0.        ,
        0.        , 0.        , 0.        , 0.        , 0.        ,
        0.        , 0.        , 0.        , 0.  

Аналогичным образом получим матрицу с весами по жанрам для какого-нибудь пользователя, например, для пользователя с идентификатором 1000010.

In [23]:
USER_ID = 1000010

user_events = events_train.loc[events_train["user_id"]==USER_ID, ["item_id", "rating"]]
user_items = items[items["item_id"].isin(user_events["item_id"])]

user_items_genres_csr = get_item2genre_matrix(genres, user_items)

In [24]:
user_items_genres_csr

<Compressed Sparse Row sparse matrix of dtype 'float64'
	with 149 stored elements and shape (22, 815)>

In [25]:
user_items_genres_csr[0].toarray()

array([[0.        , 0.5940531 , 0.        , 0.14343098, 0.        ,
        0.        , 0.        , 0.16086999, 0.        , 0.        ,
        0.        , 0.10164593, 0.        , 0.        , 0.        ,
        0.        , 0.        , 0.        , 0.        , 0.        ,
        0.        , 0.        , 0.        , 0.        , 0.        ,
        0.        , 0.        , 0.        , 0.        , 0.        ,
        0.        , 0.        , 0.        , 0.        , 0.        ,
        0.        , 0.        , 0.        , 0.        , 0.        ,
        0.        , 0.        , 0.        , 0.        , 0.        ,
        0.        , 0.        , 0.        , 0.        , 0.        ,
        0.        , 0.        , 0.        , 0.        , 0.        ,
        0.        , 0.        , 0.        , 0.        , 0.        ,
        0.        , 0.        , 0.        , 0.        , 0.        ,
        0.        , 0.        , 0.        , 0.        , 0.        ,
        0.        , 0.        , 0.        , 0.  

In [26]:
events_train.loc[events_train["user_id"]==USER_ID].shape

(22, 9)

Если посчитать средние, то фактически получим предпочтения пользователя по жанрам.

На практике часто пользователь явно указывает предпочтения в своём профиле. У нас таких данных нет, поэтому предпочтения пользователя по жанрам вычислим автоматически на основе его истории поведения. 

In [27]:
# преобразуем пользовательские оценки из списка в вектор-столбец

user_ratings = user_events["rating"].to_numpy() / 5
user_ratings = np.expand_dims(user_ratings, axis=1)

user_ratings

array([[0.4],
       [0.8],
       [0.8],
       [0.8],
       [0.6],
       [0.8],
       [0.8],
       [0.6],
       [1. ],
       [0.2],
       [0.8],
       [0.4],
       [0.8],
       [0.8],
       [0.8],
       [1. ],
       [0.6],
       [0.8],
       [1. ],
       [1. ],
       [1. ],
       [0.2]])

In [28]:
user_items_genres_weighted = user_items_genres_csr.multiply(user_ratings)

user_genres_scores = np.asarray(user_items_genres_weighted.mean(axis=0))

In [29]:
user_genres_scores.shape

(1, 815)

In [30]:
user_genres_scores

array([[0.0742606 , 0.19525295, 0.09668675, 0.02012494, 0.00271657,
        0.04435858, 0.02865144, 0.03016855, 0.04590192, 0.        ,
        0.00210007, 0.01746508, 0.00032864, 0.00200323, 0.        ,
        0.01029832, 0.        , 0.        , 0.        , 0.02003393,
        0.        , 0.        , 0.01112404, 0.00322057, 0.        ,
        0.        , 0.00528719, 0.00799514, 0.00157504, 0.01568879,
        0.        , 0.0025671 , 0.        , 0.        , 0.        ,
        0.        , 0.        , 0.00639448, 0.00218746, 0.00611828,
        0.        , 0.00409227, 0.        , 0.00632951, 0.00122303,
        0.00788988, 0.        , 0.0058798 , 0.        , 0.        ,
        0.00152893, 0.        , 0.        , 0.00142749, 0.00161081,
        0.        , 0.        , 0.        , 0.00399299, 0.00210032,
        0.00124958, 0.00628143, 0.        , 0.        , 0.0007566 ,
        0.        , 0.        , 0.        , 0.        , 0.        ,
        0.        , 0.        , 0.        , 0.00

In [31]:
# выведем список жанров, которые предпочитает пользователь

user_genres = genres.copy()
user_genres["score"] = np.ravel(user_genres_scores)
user_genres = user_genres[user_genres["score"] > 0].sort_values(by=["score"], ascending=False)

user_genres.head(5)

,name,votes,score
1,Fiction,6406698,0.195253
2,Classics,3415071,0.096687
0,Fantasy,6850115,0.074261
8,Science Fiction,1218997,0.045902
5,Nonfiction,1737798,0.044359


### Получим наиболее релевантные рекомендации для пользователя

In [32]:
# вычисляем сходство между вектором пользователя и векторами по книгам

similarity_scores = cosine_similarity(all_items_genres_csr, user_genres_scores)

In [33]:
similarity_scores.shape

(43312, 1)

In [34]:
similarity_scores

array([[0.51328292],
       [0.50496693],
       [0.53061606],
       ...,
       [0.67123638],
       [0.03087739],
       [0.30265991]])

In [35]:
# преобразуем в одномерный массив

similarity_scores = similarity_scores.flatten()

In [36]:
similarity_scores

array([0.51328292, 0.50496693, 0.53061606, ..., 0.67123638, 0.03087739,
       0.30265991])

In [37]:
# получаем индексы top-k (по убыванию значений), по сути, индексы книг (encoded)

k = 5
top_k_indices = np.argsort(similarity_scores * -1)[:5]

In [38]:
selected_items = items[items["item_id_enc"].isin(top_k_indices)]

with pd.option_context("max_colwidth", 100):
   display(selected_items[["author", "title", "genre_and_votes"]]) 

,author,title,genre_and_votes
564712,Ray Bradbury,"Farewell Summer (Green Town, #3)","{'Fiction': 170, 'Fantasy': 72, 'Science Fiction': 72, 'Classics': 52}"
1358935,John Fowles,The Magus,"{'Fiction': 1204, 'Classics': 421, 'Fantasy': 228, 'Mystery': 203, 'Literature': 167}"
80465,G.K. Chesterton,The Napoleon of Notting Hill,"{'Fiction': 166, 'Classics': 88, 'Fantasy': 44, 'Humor': 22, 'Literature': 20}"
1168335,Ray Bradbury,"Dandelion Wine (Green Town, #1)","{'Fiction': 1438, 'Classics': 914, 'Science Fiction': 529, 'Fantasy': 456, 'Young Adult': 212}"
2244467,Samuel Butler,"Erewhon (Erewhon , #1)","{'Fiction': 162, 'Classics': 139, 'Science Fiction': 60, 'Fantasy': 55}"


In [39]:
# Какой жанр преобладает в полученных рекомендациях?
 
best_genres = genres.copy()
best_genres["recs_scores"] = all_items_genres_csr[top_k_indices].toarray().sum(axis=0)
best_genres.sort_values("recs_scores", ascending=False).head(5)

,name,votes,recs_scores
1,Fiction,6406698,2.288934
2,Classics,3415071,1.181956
0,Fantasy,6850115,0.689396
8,Science Fiction,1218997,0.490008
37,Literature,191070,0.133947


---

# === Базовые подходы: валидация

In [40]:
als_recommendations_df = pd.read_parquet("als_recommendations.parquet") 

In [41]:
als_recommendations_df.shape

(43058500, 3)

In [42]:
als_recommendations_df.head()

,user_id,item_id,score
0,1000000,3,0.990941
1,1000000,15881,0.896617
2,1000000,5,0.864404
3,1000000,6,0.822254
4,1000000,2,0.774095


---

## Метрики классификации («попадания») <a id='_cls_metrics'></a>

Используются метрики, традиционные для задач бинарной классификации: precision, recall, и их производные, например, F1.

Precision вычисляется по формуле: 

$$
precision=\frac{TP}{TP+FP}
$$​

Метрика показывает, какая доля объектов, отнесённых моделью к положительному классу, действительно относится к этому классу. В контексте задачи рекомендаций это означает, что рекомендованный айтем действительно нравится пользователю (есть в тестовой выборке).

Recall вычисляется по формуле: 

$$
recall=\frac{TP}{TP+FN}
$$

Recall измеряет способность модели обнаруживать все истинные случаи в данных. В контексте задачи рекомендаций это означает, что модель смогла «найти» те айтемы, которые представлены в тестовой выборке.

Посчитаем recall и precision для ALS-рекомендаций (`als_recommendations_df`). Для этого события в тестовой выборке и рекомендации для одних и тех же пользователей разметим признаками:
- `gt` (ground truth): объект есть в тестовой выборке;
- `pr` (predicted): объект есть в рекомендациях.

Теперь разметим признаки бинарной классификации:
- `TP`: объект есть и в тестовой выборке, и в рекомендациях (истинная рекомендация),
- `FP`: объекта нет в тестовой выборке, но он есть в рекомендациях (ложноположительная рекомендация),
- `FN`: объект есть в тестовой выборке, но его нет в рекомендациях (ложноотрицательная рекомендация)

In [43]:
def process_events_recs_for_binary_metrics(events_train, events_test, recs, top_k=None):

    """
    размечает пары <user_id, item_id> для общего множества пользователей признаками
    - gt (ground truth)
    - pr (prediction)
    top_k: расчёт ведётся только для top k-рекомендаций
    """

    events_test["gt"] = True
    common_users = set(events_test["user_id"]) & set(recs["user_id"])

    print(f"Common users: {len(common_users)}")
    
    events_for_common_users = events_test[events_test["user_id"].isin(common_users)].copy()
    recs_for_common_users = recs[recs["user_id"].isin(common_users)].copy()
    
    # оставляет только те item_id, которые были в events_train, 
    # т. к. модель не имела никакой возможности давать рекомендации для новых айтемов
    events_for_common_users = events_for_common_users[events_for_common_users["item_id"].isin(events_train["item_id"].unique())]


    recs_for_common_users = recs_for_common_users.sort_values("score", ascending=False)
    if top_k is not None:
        recs_for_common_users = recs_for_common_users.groupby("user_id").head(top_k)
    
    events_recs_common = events_for_common_users[["user_id", "item_id", "gt"]].merge(
        recs_for_common_users[["user_id", "item_id", "score"]], 
        on=["user_id", "item_id"], how="outer")

    events_recs_common["gt"] = events_recs_common["gt"].fillna(False)
    events_recs_common["pr"] = events_recs_common["score"].notna()

    events_recs_common["tp"] = events_recs_common["gt"] & events_recs_common["pr"]
    events_recs_common["fp"] = ~events_recs_common["gt"] & events_recs_common["pr"]
    events_recs_common["fn"] = events_recs_common["gt"] & ~events_recs_common["pr"]

    return events_recs_common 

In [44]:
events_recs_for_binary_metrics = process_events_recs_for_binary_metrics(
    events_train,
    events_test,
    als_recommendations_df,
    top_k=5,
)

/tmp/ipykernel_17286/2927831550.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  events_test["gt"] = True


Common users: 123223


In [45]:
events_recs_for_binary_metrics.sort_values(["user_id", "score"], ascending=[True, False])

,user_id,item_id,gt,score,pr,tp,fp,fn
466868,1000003,9361589,False,1.129593,True,False,True,False
467071,1000003,7260188,False,1.128976,True,False,True,False
474918,1000003,6148028,False,1.108895,True,False,True,False
482144,1000003,2767052,False,1.092192,True,False,True,False
516999,1000003,9460487,False,1.028991,True,False,True,False
...,...,...,...,...,...,...,...,...
835683,1430584,17927395,False,0.460429,True,False,True,False
838297,1430584,16096824,False,0.454709,True,False,True,False
842294,1430584,20613470,False,0.445634,True,False,True,False
418722,1430584,23395680,True,NaN,False,False,False,True


In [46]:
def compute_cls_metrics(events_recs_for_binary_metric):
    
    groupper = events_recs_for_binary_metric.groupby("user_id")

    # precision = tp / (tp + fp)
    precision = groupper["tp"].sum()/(groupper["tp"].sum()+groupper["fp"].sum())
    precision = precision.fillna(0).mean()
    
    # recall = tp / (tp + fn)
    recall = groupper["tp"].sum()/(groupper["tp"].sum()+groupper["fn"].sum())
    recall = recall.fillna(0).mean()

    return precision, recall 

In [49]:
def display_metrics(precision, recall, k=5):
    print(f"precision@{k} = {precision}")
    print(f"recall@{k} = {recall}")
    print(f"f1@{k} = {2*precision*recall/(precision+recall)}")

Получим значения метрик precision@5, recall@5.

In [50]:
precision_5, recall_5 = compute_cls_metrics(events_recs_for_binary_metrics)

display_metrics(precision_5, recall_5, 5)

precision@5 = 0.007612215252022756
recall@5 = 0.01415978860714534
f1@5 = 0.009901464238014182


Посчитаем метрики precision@10, recall@10

In [52]:
precision, recall = compute_cls_metrics(
    process_events_recs_for_binary_metrics(
        events_train,
        events_test,
        als_recommendations_df,
        top_k=10,
    )
)

print()
display_metrics(precision, recall, 10)

/tmp/ipykernel_17286/2927831550.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  events_test["gt"] = True


Common users: 123223

precision@10 = 0.008751612929404413
recall@10 = 0.03134621206486989
f1@10 = 0.013683032176131269


Посчитаем полные метрики precision, recall

In [53]:
precision, recall = compute_cls_metrics(
    process_events_recs_for_binary_metrics(
        events_train,
        events_test,
        als_recommendations_df,
        top_k=None,
    )
)

print()
display_metrics(precision, recall, "")

/tmp/ipykernel_17286/2927831550.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  events_test["gt"] = True


Common users: 123223

precision@ = 0.0065323843762933875
recall@ = 0.20423433248344025
f1@ = 0.012659846701558763


## Метрики ранжирования

[Оценить полученные рекомендации.](#_ngdc)

---

# === Двухстадийный подход: метрики

In [9]:
als_recommendations_df = pd.read_parquet("als_recommendations.parquet") 

In [10]:
als_recommendations_df.shape

(43058500, 3)

In [11]:
als_recommendations_df.head()

,user_id,item_id,score
0,1000000,3,0.990941
1,1000000,15881,0.896617
2,1000000,5,0.864404
3,1000000,6,0.822254
4,1000000,2,0.774095


---

## Специфичные метрики

### Покрытие (англ. coverage)

Рекомендательная система может выдавать точные рекомендации, но только для небольшого подмножества объектов или подмножества пользователей. Формально метрики будут иметь высокие значения, а фактически рекомендации будут малоценными для большинства пользователей.

Чтобы оценить рекомендации по этому фактору, используют метрику «покрытие» (англ. Coverage), которая отвечает на вопрос, насколько полно объекты (`items`) или пользователи (`users`) покрыты рекомендациями. 

Так, покрытие по объектам можно оценить как долю уникальных объектов в рекомендациях (от всех объектов):

$$
Coverage_I = \frac{\sum_{i=1}^{n} {
  \begin{cases}
    1  & \quad \text{, если объект i есть в рекомендациях}\\
    0  & \quad \text{, в прочих случаях}
  \end{cases}
}}{n}
$$

где *n* — количество всех объектов.

Аналогичным образом можно определить данную метрику и для пользователей — речь о полноте представленных для них рекомендаций:

$$
Coverage_U = \frac{\sum_{i=1}^{m} {
  \begin{cases}
    1  & \quad \text{, если для пользователя i есть рекомендации}\\
    0  & \quad \text{, в прочих случаях}
  \end{cases}
}}{m}
$$

где *m* — количество всех пользователей.

In [12]:
cov_items = np.isin(
    als_recommendations_df["item_id"].unique(),
    items["item_id"].unique(),
).sum() / items["item_id"].nunique()

print(f"{cov_items:.2f}") 

0.09


### Новизна (англ. novelty)

В простом случае новизну можно оценить как долю объектов, ранее не виденных пользователем, среди топ-k рекомендованных:

$$
Novelty@k = \frac{∣unknown@k∣}{∣recommended@k∣}
$$

In [13]:
als_recommendations_df = als_recommendations_df.merge(
    events_train[["user_id", "item_id", "is_read"]], on=["user_id", "item_id"], how="left"
)

als_recommendations_df["is_read"] = als_recommendations_df["is_read"].fillna(False).astype("bool")

In [14]:
als_recommendations_df.head()

,user_id,item_id,score,is_read
0,1000000,3,0.990941,True
1,1000000,15881,0.896617,True
2,1000000,5,0.864404,True
3,1000000,6,0.822254,False
4,1000000,2,0.774095,True


In [15]:
# проставим ранги

als_recommendations_df = als_recommendations_df.sort_values("score", ascending=False)
als_recommendations_df["rank"] = als_recommendations_df.groupby("user_id").cumcount() + 1

In [16]:
# посчитаем novelty по пользователям

novelty_5 = (1-als_recommendations_df[als_recommendations_df["rank"] <= 5].groupby("user_id")["is_read"].mean())

In [17]:
novelty_5

user_id
1000000    0.2
1000001    0.8
1000002    0.8
1000003    0.0
1000004    0.0
          ... 
1430580    0.8
1430581    0.8
1430582    0.8
1430583    1.0
1430584    0.8
Name: is_read, Length: 430585, dtype: float64

In [18]:
# посчитаем средний novelty

novelty_5.mean()

0.607333279143491

## Разнообразие (англ. diversity)

Разнообразие можно посчитать как:

$$
Diversity@k = \frac{∣количество разных значений выбранного фактора@k∣}{∣recommended@k∣}
$$

---

# === Двухстадийный подход: модель

## Разбиение данных

Используем отложенную тестовую часть данных — назовём её `events_test` — для получения двух новых частей данных:
- одна, составляющая первые 45 дней, будет использоваться для таргетов,
- другая, состоящая из 45 последних дней, будет новой тестовой выборкой.

In [9]:
# задаём точку разбиения
split_date_for_labels = pd.to_datetime("2017-09-15").date()

split_date_for_labels_idx = events_test["started_at"] < split_date_for_labels
events_labels = events_test[split_date_for_labels_idx].copy()
events_test_2 = events_test[~split_date_for_labels_idx].copy() 

In [10]:
events_labels.shape, events_test_2.shape

((253765, 9), (171197, 9))

In [11]:
# Сколько уникальных пользователей есть в events_labels?

events_labels["user_id"].nunique(), events_test_2["user_id"].nunique()

(99849, 75194)

## Подготовка кандидатов для обучения

In [12]:
# загружаем рекомендации от двух базовых генераторов

als_recommendations = pd.read_parquet("candidates/training/als_recommendations.parquet")
content_recommendations = pd.read_parquet("candidates/training/content_recommendations.parquet")

In [13]:
als_recommendations.shape

(43058500, 3)

In [14]:
als_recommendations.head()

,user_id,item_id,score
0,1000000,3,0.972557
1,1000000,15881,0.890201
2,1000000,5,0.865850
3,1000000,6,0.834282
4,1000000,2,0.792929


In [15]:
als_recommendations.groupby("user_id")["item_id"].nunique().value_counts()

item_id
100    430585
Name: count, dtype: int64

In [16]:
content_recommendations.shape

(42822000, 3)

In [17]:
content_recommendations.head()

,user_id,item_id,score
0,1000000,1,0.933434
1,1000000,2,0.925806
2,1000000,3,0.920225
3,1000000,5,0.918026
4,1000000,6,0.916345


In [18]:
content_recommendations.groupby("user_id")["item_id"].nunique().value_counts()

item_id
100    428220
Name: count, dtype: int64

Объединение:

In [19]:
# 30 sec

candidates = pd.merge(
    als_recommendations[["user_id", "item_id", "score"]].rename(columns={"score": "als_score"}),
    content_recommendations[["user_id", "item_id", "score"]].rename(columns={"score": "cnt_score"}),
    on=["user_id", "item_id"],
    how="outer"
) 

In [20]:
# (82993094, 4)
candidates.shape

(82993094, 4)

In [21]:
candidates.head()

,user_id,item_id,als_score,cnt_score
0,1000000,3,0.972557,0.920225
1,1000000,15881,0.890201,0.905740
2,1000000,5,0.865850,0.918026
3,1000000,6,0.834282,0.916345
4,1000000,2,0.792929,0.925806


## Таргеты

При этом следует контролировать баланс классов и при необходимости проводить негативное семплирование: уменьшать количество строк с отрицательным таргетом, чтобы не было сильного дисбаланса классов.

Для негативного семплирования возможны различные стратегии: отбрасывание случайных записей равномерно, пропорционально их популярности, с различными весами и так далее.

In [22]:
# добавляем таргет к кандидатам со значением:
# — 1 для тех item_id, которые пользователь прочитал
# — 0, для всех остальных 
# 30 sec

events_labels["target"] = 1
candidates = candidates.merge(
    events_labels[["user_id", "item_id", "target"]],
    on=["user_id", "item_id"],
    how="left"
)

candidates["target"] = candidates["target"].fillna(0).astype("int")

In [23]:
# в кандидатах оставляем только тех пользователей, у которых есть хотя бы один положительный таргет
# 30 sec

candidates_to_sample = candidates.groupby("user_id").filter(lambda x: x["target"].sum() > 0)

In [24]:
# для каждого пользователя оставляем только 4 негативных примера

negatives_per_user = 4
candidates_for_train = pd.concat([
    candidates_to_sample[candidates_to_sample["target"]==1],
    candidates_to_sample[candidates_to_sample["target"]==0] \
        .groupby("user_id") \
        .apply(lambda x: x.sample(negatives_per_user, random_state=0))
])

In [25]:
# (213708, 5)
candidates_for_train.shape

(213708, 5)

In [26]:
candidates_for_train.head()

,user_id,item_id,als_score,cnt_score,target
615,1000006,29868610,0.286715,NaN,1
632,1000006,7445,0.230529,NaN,1
649,1000006,18812405,0.178382,NaN,1
1998,1000019,37415,0.043595,NaN,1
2302,1000023,7260188,0.598791,NaN,1


## Обучение модели

In [27]:
# задаём имена колонок признаков и таргета

features = ['als_score', 'cnt_score']
target = 'target'

In [28]:
# Create the Pool object

train_data = Pool(
    data=candidates_for_train[features], 
    label=candidates_for_train[target]
)

In [29]:
# инициализируем модель CatBoostClassifier

cb_model = CatBoostClassifier(
    iterations=1000,
    learning_rate=0.1,
    depth=6,
    loss_function='Logloss',
    verbose=100,
    random_seed=0
)

In [30]:
# тренируем модель
# 20 sec

cb_model.fit(train_data) 

0:	learn: 0.6526057	total: 70.5ms	remaining: 1m 10s
100:	learn: 0.5118959	total: 1.97s	remaining: 17.6s
200:	learn: 0.5111710	total: 3.91s	remaining: 15.5s
300:	learn: 0.5105208	total: 5.86s	remaining: 13.6s
400:	learn: 0.5100174	total: 7.81s	remaining: 11.7s
500:	learn: 0.5095747	total: 9.75s	remaining: 9.71s
600:	learn: 0.5091600	total: 11.7s	remaining: 7.77s
700:	learn: 0.5087803	total: 13.7s	remaining: 5.84s
800:	learn: 0.5084220	total: 15.7s	remaining: 3.89s
900:	learn: 0.5080930	total: 17.6s	remaining: 1.94s
999:	learn: 0.5078081	total: 19.5s	remaining: 0us


## Подготовка кандидатов для рекомендаций

Представим, что натренированная модель используется только некоторое время спустя, когда уже появились новые рекомендации (кандидаты) от базовых генераторов, обученных на объединении событий из `events_train` и `events_label`.

Иными словами, когда события из `events_label` уже стали частью тренировочного набора данных.

Эти новые рекомендации были заранее подготовлены и сохранены в файлах `als_recommendations.parquet` и `content_recommendations.parquet` в директории `candidates/inference`.

Используем их для составления нового списка кандидатов `candidates_to_rank`, который понадобится готовой ранжирующей модели. 

In [31]:
# загружаем рекомендации от двух базовых генераторов

als_recommendations_2 = pd.read_parquet("candidates/inference/als_recommendations.parquet")
content_recommendations_2 = pd.read_parquet("candidates/inference/content_recommendations.parquet")

Объединение:

In [32]:
# 40 sec

candidates_to_rank = pd.merge(
    als_recommendations_2[["user_id", "item_id", "score"]].rename(columns={"score": "als_score"}),
    content_recommendations_2[["user_id", "item_id", "score"]].rename(columns={"score": "cnt_score"}),
    on=["user_id", "item_id"],
    how="outer"
)

In [33]:
# оставляем только тех пользователей, что есть в тестовой выборке, для экономии ресурсов

candidates_to_rank = candidates_to_rank[candidates_to_rank["user_id"].isin(events_test_2["user_id"].unique())]

In [34]:
# (14517152, 4)

candidates_to_rank.shape

(14517152, 4)

In [35]:
candidates_to_rank.head()

,user_id,item_id,als_score,cnt_score
300,1000003,7260188,1.129979,NaN
301,1000003,6148028,1.123475,NaN
302,1000003,2767052,1.112699,NaN
303,1000003,9361589,1.060634,NaN
304,1000003,9969571,0.903286,NaN


## Ранжирование кандидатов для рекомендаций

In [36]:
inference_data = Pool(data=candidates_to_rank[features])

predictions = cb_model.predict_proba(inference_data)

candidates_to_rank["cb_score"] = predictions[:, 1]

In [37]:
candidates_to_rank.shape

(14517152, 5)

In [38]:
candidates_to_rank.head()

,user_id,item_id,als_score,cnt_score,cb_score
300,1000003,7260188,1.129979,NaN,0.509005
301,1000003,6148028,1.123475,NaN,0.509005
302,1000003,2767052,1.112699,NaN,0.509005
303,1000003,9361589,1.060634,NaN,0.363649
304,1000003,9969571,0.903286,NaN,0.376137


In [39]:
# отберем топ рекоменадаций

candidates_to_rank = candidates_to_rank.sort_values(["user_id", "cb_score"], ascending=[True, False])
candidates_to_rank["rank"] = candidates_to_rank.groupby("user_id").cumcount() + 1

max_recommendations_per_user = 100

# final_recommendations = candidates_to_rank.groupby("user_id").head(max_recommendations_per_user)
final_recommendations = candidates_to_rank[candidates_to_rank["rank"] <= max_recommendations_per_user]

In [40]:
final_recommendations.shape

(7519400, 6)

In [41]:
final_recommendations.head()

,user_id,item_id,als_score,cnt_score,cb_score,rank
347,1000003,49628,0.446143,0.906649,0.583617,1
300,1000003,7260188,1.129979,NaN,0.509005,2
301,1000003,6148028,1.123475,NaN,0.509005,3
302,1000003,2767052,1.112699,NaN,0.509005,4
320,1000003,43641,0.617602,NaN,0.477032,5


## Валидация

[Метрики классификации](#_cls_metrics)

In [42]:
def process_events_recs_for_binary_metrics(events_train, events_test, recs, top_k=None):

    """
    размечает пары <user_id, item_id> для общего множества пользователей признаками
    - gt (ground truth)
    - pr (prediction)
    top_k: расчёт ведётся только для top k-рекомендаций
    """

    events_test["gt"] = True
    common_users = set(events_test["user_id"]) & set(recs["user_id"])

    print(f"Common users: {len(common_users)}")
    
    events_for_common_users = events_test[events_test["user_id"].isin(common_users)].copy()
    recs_for_common_users = recs[recs["user_id"].isin(common_users)].copy()
    
    # оставляет только те item_id, которые были в events_train, 
    # т. к. модель не имела никакой возможности давать рекомендации для новых айтемов
    events_for_common_users = events_for_common_users[events_for_common_users["item_id"].isin(events_train["item_id"].unique())]


    recs_for_common_users = recs_for_common_users.sort_values("score", ascending=False)
    if top_k is not None:
        recs_for_common_users = recs_for_common_users.groupby("user_id").head(top_k)
    
    events_recs_common = events_for_common_users[["user_id", "item_id", "gt"]].merge(
        recs_for_common_users[["user_id", "item_id", "score"]], 
        on=["user_id", "item_id"], how="outer")

    events_recs_common["gt"] = events_recs_common["gt"].fillna(False)
    events_recs_common["pr"] = events_recs_common["score"].notna()

    events_recs_common["tp"] = events_recs_common["gt"] & events_recs_common["pr"]
    events_recs_common["fp"] = ~events_recs_common["gt"] & events_recs_common["pr"]
    events_recs_common["fn"] = events_recs_common["gt"] & ~events_recs_common["pr"]

    return events_recs_common 

In [43]:
def compute_cls_metrics(events_recs_for_binary_metric):
    
    groupper = events_recs_for_binary_metric.groupby("user_id")

    # precision = tp / (tp + fp)
    precision = groupper["tp"].sum()/(groupper["tp"].sum()+groupper["fp"].sum())
    precision = precision.fillna(0).mean()
    
    # recall = tp / (tp + fn)
    recall = groupper["tp"].sum()/(groupper["tp"].sum()+groupper["fn"].sum())
    recall = recall.fillna(0).mean()

    return precision, recall 

In [44]:
def display_metrics(precision, recall, k=5):
    print(f"precision@{k} = {precision}")
    print(f"recall@{k} = {recall}")
    print(f"f1@{k} = {2*precision*recall/(precision+recall)}")

In [45]:
# для экономии ресурсов оставим события только тех пользователей, 
# для которых следует оценить рекомендации

events_inference = pd.concat([events_train, events_labels])
events_inference = events_inference[events_inference["user_id"].isin(events_test_2["user_id"].unique())]

In [46]:
precision, recall = compute_cls_metrics(
    process_events_recs_for_binary_metrics(
        events_inference,
        events_test_2,
        final_recommendations.rename(columns={"cb_score": "score"}),
        top_k=5,
    )
)

print()
display_metrics(precision, recall, 5)

Common users: 75194

precision@5 = 0.006293055296965184
recall@5 = 0.015338055044339712
f1@5 = 0.008924482101837666


---

# === Двухстадийный подход: построение признаков

## Признаки объектов

Посчитаем новый признак — «возраст» книги на основе года публикации. Назовём его `age`.

In [47]:
items["age"] = 2018-items["publication_year"]
invalid_age_idx = items["age"] < 0
items.loc[invalid_age_idx, "age"] = np.nan
items["age"] = items["age"].astype("float")

Добавим признаки возраста `age` и средней популярности `average_rating` к кандидатам для тренировки модели `candidates_for_train` и к кандидатам для ранжирования `candidates_to_rank`.

In [48]:
candidates_for_train = candidates_for_train.merge(
    items[["item_id", "age"]],
    on="item_id",
    how="left"
)
candidates_to_rank = candidates_to_rank.merge(
    items[["item_id", "age"]],
    on="item_id",
    how="left"
)

In [49]:
# Какой медианный возраст книги получился для candidates_to_rank?

candidates_to_rank["age"].median()

7.0

## Признаки пользователей

In [50]:
def get_user_features(events):
    """ считает пользовательские признаки """
    
    user_features = events.groupby("user_id").agg(
        reading_years=("started_at", lambda x: (x.max()-x.min()).days/365.25),
        books_read=("item_id", "count"),
        rating_avg=("rating", "mean"),
        rating_std=("rating", "std")
    )
    
    user_features["books_per_year"] = user_features["books_read"] / user_features["reading_years"]
    
    return user_features

In [51]:
# 30 sec

user_features_for_train = get_user_features(events_train)
candidates_for_train = candidates_for_train.merge(user_features_for_train, on="user_id", how="left")

In [52]:
user_features_for_ranking = get_user_features(events_inference)
candidates_to_rank = candidates_to_rank.merge(user_features_for_ranking, on="user_id", how="left")

In [53]:
# Какой вышла медиана количества прочитанных книг по всем кандидатам в `candidates_for_train`?

candidates_for_train["books_read"].median()

32.0

## Парные признаки

[Контент для рекомендаций](#_get_genres)

### Перекодировать идентификаторы пользователей и объектов.

In [54]:
# перекодируем идентификаторы объектов: 
# из имеющихся в последовательность 0, 1, 2, ...

item_encoder = LabelEncoder()
item_encoder.fit(items["item_id"])

items["item_id_enc"] = item_encoder.transform(items["item_id"])

### Составим список всех жанров <a id='_get_genres_'></a>

In [55]:
# Сначала преобразуем значения в genre_and_votes из текстового представления в тип в Python:

items["genre_and_votes"] = items["genre_and_votes"].apply(eval) 

Составим список жанров с долями голосов по ним

In [56]:
def get_genres(genre_and_votes):

    """ 
    извлекает список жанров по всем книгам, 
    подсчитывает долю голосов по каждому их них
    """
    
    genres_counter = defaultdict(int)
    
    for dict_votes in genre_and_votes.to_list():
        if dict_votes is None or not isinstance(dict_votes, dict):
            continue
        for genre, votes in dict_votes.items():
            # увеличиваем счётчик жанров
            genres_counter[genre] += int(votes)

    genres_df = (
        pd.Series(genres_counter, name="votes")
        .sort_values(ascending=False)
        .to_frame()
        .reset_index()
        .rename(columns={"index": "name"})
    )
    # genres_df.index.name = "genre_id"
    
    return genres_df

In [57]:
genres = get_genres(items["genre_and_votes"])

In [58]:
genres.shape

(815, 2)

In [59]:
genres.head()

,name,votes
0,Fantasy,6850115
1,Fiction,6406698
2,Classics,3415071
3,Young Adult,3297027
4,Romance,2422690


### Подготовим матрицы

In [60]:
def get_item2genre_matrix(genres, items):

    '''Функция строит матрицу вида «книга-жанр».
    Индексы строк не соовтетствуют исходным интексам!!! - это новые индексы !!! Поскольку важны жанры
    '''

    genre_names_to_id = genres.reset_index().set_index("name")["index"].to_dict()
    
    # list to build CSR matrix
    genres_csr_data = []
    genres_csr_row_idx = []
    genres_csr_col_idx = []
    
    for item_idx, (k, v) in enumerate(items.iterrows()):
        if v["genre_and_votes"] is None:
            continue
        for genre_name, votes in v["genre_and_votes"].items():
            genre_idx = genre_names_to_id[genre_name]
            genres_csr_data.append(int(votes))
            genres_csr_row_idx.append(item_idx)
            genres_csr_col_idx.append(genre_idx)

    genres_csr = scipy.sparse.csr_matrix(
        (genres_csr_data, (genres_csr_row_idx, genres_csr_col_idx)),
        shape=(len(items), len(genres))
    )
    
    # нормализуем, чтобы сумма оценок принадлежности к жанру была равна 1
    genres_csr = normalize(genres_csr, norm='l1', axis=1)
    
    return genres_csr 

In [61]:
items = items.sort_values(by="item_id_enc")
print(items.shape)
items.tail()

(43312, 21)


,item_id,author,title,description,genre_and_votes,num_pages,average_rating,ratings_count,text_reviews_count,publisher,...,country_code,language_code,format,is_ebook,isbn,isbn13,genre_and_votes_dict,genre_and_votes_str,age,item_id_enc
1229539,36411879,Lylah James,The Mafia And His Angel: Part 2 (Tainted Heart...,Ayla\nThe darkness never truly left me. It's a...,"{'Romance': 38, 'Dark': 37, 'Sociology-Abuse':...",<NA>,4.64,408,87,,...,US,,,True,,,"{'Academic': None, 'Academic-Academia': None, ...","Romance 38, Dark 37, Sociology-Abuse 17",NaN,43307
534649,36421066,David Anderson,The Remnant,"Will, Kevin, Rose, Quentin and Fiona have over...",{'Science Fiction Fantasy': 1},<NA>,5.00,2,1,BWL Publishing Inc.,...,US,eng,ebook,True,,9781773628875,"{'Academic': None, 'Academic-Academia': None, ...",Science Fiction Fantasy 1user,1.0,43308
1541052,36430456,Elaine Williams Crockett,Do Not Ask,The President's beautiful twin daughters disap...,"{'Fiction': 4, 'Mystery': 3, 'Thriller': 2}",<NA>,5.00,3,3,BookBaby,...,US,,ebook,True,,9781543910445,"{'Academic': None, 'Academic-Academia': None, ...","Fiction 4, Mystery 3, Thriller 2",1.0,43309
2251007,36515426,Logan Chance,Heartbreaker,They call me a heartbreaker.\nI've constructed...,"{'Romance': 20, 'Contemporary': 6}",<NA>,4.48,185,49,,...,US,,,True,,,"{'Academic': None, 'Academic-Academia': None, ...","Romance 20, Contemporary 6",NaN,43310
96267,36524503,Emily Lloyd-Jones,The Bone Houses,A historical horror-fairy tale that follows a ...,"{'Fantasy': 841, 'Young Adult': 437, 'Horror':...",<NA>,2.00,1,1,"Little, Brown Books for Young Readers",...,US,,,False,,,"{'Academic': None, 'Academic-Academia': None, ...","Fantasy 841, Young Adult 437, Horror 356, Hist...",NaN,43311


In [62]:
# Проверка, что декодированные индексы будут соовтетствовать новым индексам:
assert items.iloc[-1]["item_id_enc"] == items.shape[0] - 1

all_items_genres_csr = get_item2genre_matrix(genres, items)

In [63]:
all_items_genres_csr

<Compressed Sparse Row sparse matrix of dtype 'float64'
	with 210895 stored elements and shape (43312, 815)>

---

In [64]:
# определяем индексы топ-10 жанров и всех остальных

genres_top_k = 10
genres_top_idx = genres.sort_values("votes", ascending=False).head(genres_top_k).index
genres_others_idx = list(set(genres.index) - set(genres_top_idx))

In [65]:
genres_top_idx

Index([0, 1, 2, 3, 4, 5, 6, 7, 8, 9], dtype='int64')

In [66]:
genres_top_columns = [f"genre_{id}" for id in genres_top_idx]
genres_others_column = "genre_others"
genre_columns = genres_top_columns + [genres_others_column]

genre_columns

['genre_0',
 'genre_1',
 'genre_2',
 'genre_3',
 'genre_4',
 'genre_5',
 'genre_6',
 'genre_7',
 'genre_8',
 'genre_9',
 'genre_others']

In [67]:
# составляем таблицу принадлежности книг к жанрам

item_genres = (
    pd.concat([
        # топ жанров
        pd.DataFrame(all_items_genres_csr[:, genres_top_idx].toarray(), columns=genres_top_columns),
        # все остальные жанры
        pd.DataFrame(all_items_genres_csr[:, genres_others_idx].sum(axis=1), columns=[genres_others_column])
        ],
        axis=1)
    .reset_index()
    .rename(columns={"index": "item_id_enc"})
)

In [68]:
item_genres.shape

(43312, 12)

In [69]:
item_genres.head()

,item_id_enc,genre_0,genre_1,genre_2,genre_3,genre_4,genre_5,genre_6,genre_7,genre_8,genre_9,genre_others
0,0,0.524988,0.148026,0.014064,0.170655,0.0,0.0,0.0,0.0,0.0,0.0,0.142267
1,1,0.621931,0.174786,0.000000,0.203283,0.0,0.0,0.0,0.0,0.0,0.0,0.000000
2,2,0.514586,0.154140,0.022797,0.153916,0.0,0.0,0.0,0.0,0.0,0.0,0.154561
3,3,0.518708,0.148702,0.015347,0.160382,0.0,0.0,0.0,0.0,0.0,0.0,0.156860
4,4,0.623564,0.176369,0.000000,0.200067,0.0,0.0,0.0,0.0,0.0,0.0,0.000000


In [70]:
# объединяем информацию принадлежности книг к жанрам с основной информацией о книгах

items = items.merge(item_genres, on="item_id_enc", how="left")

In [71]:
def get_user_genres(events, items, item_genre_columns):
    user_genres = (
        events
        .merge(items[["item_id"] + item_genre_columns], on="item_id", how="left")
        .groupby("user_id")[item_genre_columns].mean()
    )
    return user_genres

In [72]:
user_genres_for_train = get_user_genres(events_train, items, genre_columns)
candidates_for_train = candidates_for_train.merge(user_genres_for_train, on="user_id", how="left")

user_genres_for_ranking = get_user_genres(events_inference, items, genre_columns)
candidates_to_rank = candidates_to_rank.merge(user_genres_for_ranking, on="user_id", how="left") 


In [73]:
# Какой получилось медиана жанровости книг в candidates_for_train для жанра “Romance”? Ответ округлите до сотых.

candidates_for_train[f'genre_{genres[genres["name"]=="Romance"].index[0]}'].median()

0.038488976462249

## Обучение и получение рекомендаций

In [74]:
candidates_for_train.shape

(213708, 22)

In [75]:
candidates_for_train.head()

,user_id,item_id,als_score,cnt_score,target,age,reading_years,books_read,rating_avg,rating_std,...,genre_1,genre_2,genre_3,genre_4,genre_5,genre_6,genre_7,genre_8,genre_9,genre_others
0,1000006,29868610,0.286715,NaN,1,NaN,1.820671,17.0,4.294118,0.685994,...,0.246138,0.105182,0.057684,0.010375,0.078927,0.004294,0.021665,0.008603,0.0,0.286282
1,1000006,7445,0.230529,NaN,1,12.0,1.820671,17.0,4.294118,0.685994,...,0.246138,0.105182,0.057684,0.010375,0.078927,0.004294,0.021665,0.008603,0.0,0.286282
2,1000006,18812405,0.178382,NaN,1,4.0,1.820671,17.0,4.294118,0.685994,...,0.246138,0.105182,0.057684,0.010375,0.078927,0.004294,0.021665,0.008603,0.0,0.286282
3,1000019,37415,0.043595,NaN,1,12.0,0.276523,6.0,4.166667,1.169045,...,0.158224,0.000000,0.000000,0.000000,0.195082,0.000000,0.000000,0.082617,0.0,0.514445
4,1000023,7260188,0.598791,NaN,1,8.0,0.005476,2.0,3.500000,0.707107,...,0.170366,0.000000,0.159612,0.019622,0.000000,0.000000,0.000000,0.077326,0.0,0.254950


In [76]:
candidates_for_train.columns

Index(['user_id', 'item_id', 'als_score', 'cnt_score', 'target', 'age',
       'reading_years', 'books_read', 'rating_avg', 'rating_std',
       'books_per_year', 'genre_0', 'genre_1', 'genre_2', 'genre_3', 'genre_4',
       'genre_5', 'genre_6', 'genre_7', 'genre_8', 'genre_9', 'genre_others'],
      dtype='object')

In [77]:
# задаём имена колонок признаков и таргета

features = [
    'als_score', 'cnt_score', 
    'age', 'reading_years', 'books_read', 
    'rating_avg', 'rating_std', 'books_per_year'
] + genre_columns

target = 'target'

In [78]:
# создаём Pool

train_data = Pool(
    data=candidates_for_train[features], 
    label=candidates_for_train[target]
)

In [79]:
# инициализируем модель CatBoostClassifier

cb_model = CatBoostClassifier(
    iterations=1000,
    learning_rate=0.1,
    depth=6,
    loss_function='Logloss',
    verbose=100,
    random_seed=0,
)

In [80]:
# тренируем модель
# 30 sec

cb_model.fit(train_data)

0:	learn: 0.6463996	total: 29.3ms	remaining: 29.3s
100:	learn: 0.4710013	total: 2.67s	remaining: 23.8s
200:	learn: 0.4644069	total: 5.3s	remaining: 21.1s
300:	learn: 0.4595792	total: 7.87s	remaining: 18.3s
400:	learn: 0.4556840	total: 10.4s	remaining: 15.6s
500:	learn: 0.4522164	total: 13s	remaining: 12.9s
600:	learn: 0.4489011	total: 15.6s	remaining: 10.4s
700:	learn: 0.4458227	total: 18.2s	remaining: 7.75s
800:	learn: 0.4428398	total: 20.8s	remaining: 5.16s
900:	learn: 0.4401834	total: 23.3s	remaining: 2.56s
999:	learn: 0.4376329	total: 25.9s	remaining: 0us


Получим топ-100 самых релевантных рекомендация для каждого пользователя, используя обученную модель.

In [81]:
inference_data = Pool(data=candidates_to_rank[features])

predictions = cb_model.predict_proba(inference_data)

candidates_to_rank["cb_score"] = predictions[:, 1]

In [82]:
candidates_to_rank.shape

(14517152, 23)

In [83]:
candidates_to_rank.head()

,user_id,item_id,als_score,cnt_score,cb_score,rank,age,reading_years,books_read,rating_avg,...,genre_1,genre_2,genre_3,genre_4,genre_5,genre_6,genre_7,genre_8,genre_9,genre_others
0,1000003,49628,0.446143,0.906649,0.235611,1,14.0,7.4141,94.0,3.287234,...,0.273534,0.04293,0.044853,0.013835,0.050522,0.09458,0.045009,0.04636,0.003504,0.316763
1,1000003,7260188,1.129979,NaN,0.961924,2,8.0,7.4141,94.0,3.287234,...,0.273534,0.04293,0.044853,0.013835,0.050522,0.09458,0.045009,0.04636,0.003504,0.316763
2,1000003,6148028,1.123475,NaN,0.947655,3,9.0,7.4141,94.0,3.287234,...,0.273534,0.04293,0.044853,0.013835,0.050522,0.09458,0.045009,0.04636,0.003504,0.316763
3,1000003,2767052,1.112699,NaN,0.960339,4,10.0,7.4141,94.0,3.287234,...,0.273534,0.04293,0.044853,0.013835,0.050522,0.09458,0.045009,0.04636,0.003504,0.316763
4,1000003,43641,0.617602,NaN,0.265405,5,11.0,7.4141,94.0,3.287234,...,0.273534,0.04293,0.044853,0.013835,0.050522,0.09458,0.045009,0.04636,0.003504,0.316763


In [84]:
# отберем топ рекоменадаций

candidates_to_rank = candidates_to_rank.sort_values(["user_id", "cb_score"], ascending=[True, False])
candidates_to_rank["rank"] = candidates_to_rank.groupby("user_id").cumcount() + 1

max_recommendations_per_user = 100

# final_recommendations = candidates_to_rank.groupby("user_id").head(max_recommendations_per_user)
final_recommendations = candidates_to_rank[candidates_to_rank["rank"] <= max_recommendations_per_user]

In [85]:
# (7519400, 22)
final_recommendations.shape

(7519400, 23)

In [86]:
final_recommendations.head()

,user_id,item_id,als_score,cnt_score,cb_score,rank,age,reading_years,books_read,rating_avg,...,genre_1,genre_2,genre_3,genre_4,genre_5,genre_6,genre_7,genre_8,genre_9,genre_others
1,1000003,7260188,1.129979,NaN,0.961924,1,8.0,7.4141,94.0,3.287234,...,0.273534,0.04293,0.044853,0.013835,0.050522,0.09458,0.045009,0.04636,0.003504,0.316763
3,1000003,2767052,1.112699,NaN,0.960339,2,10.0,7.4141,94.0,3.287234,...,0.273534,0.04293,0.044853,0.013835,0.050522,0.09458,0.045009,0.04636,0.003504,0.316763
2,1000003,6148028,1.123475,NaN,0.947655,3,9.0,7.4141,94.0,3.287234,...,0.273534,0.04293,0.044853,0.013835,0.050522,0.09458,0.045009,0.04636,0.003504,0.316763
98,1000003,9361589,1.060634,NaN,0.562895,4,7.0,7.4141,94.0,3.287234,...,0.273534,0.04293,0.044853,0.013835,0.050522,0.09458,0.045009,0.04636,0.003504,0.316763
28,1000003,22557272,0.540374,NaN,0.469000,5,3.0,7.4141,94.0,3.287234,...,0.273534,0.04293,0.044853,0.013835,0.050522,0.09458,0.045009,0.04636,0.003504,0.316763


In [87]:
# Сколько пользователей попало в final_recommendations?
# 75194

final_recommendations["user_id"].nunique()

75194

### Сохраним полученные рекомендации в файл, они ещё нам пригодятся.

In [88]:
final_recommendations.to_parquet("final_recommendations_feat.parquet") 

## Валидация

In [89]:
precision, recall = compute_cls_metrics(
    process_events_recs_for_binary_metrics(
        events_inference,
        events_test_2,
        final_recommendations.rename(columns={"cb_score": "score"}),
        top_k=5,
    )
)

print()
display_metrics(precision, recall, 5)

Common users: 75194

precision@5 = 0.009500758039205257
recall@5 = 0.024567175055267693
f1@5 = 0.013702433033412549


## Проверка важности признаков

In [90]:
feature_importance = pd.DataFrame(cb_model.get_feature_importance(), 
    index=features, 
    columns=["fi"]
).sort_values("fi", ascending=False)

print(feature_importance) 

                       fi
als_score       31.810044
age             21.707668
books_read       8.889603
cnt_score        4.123602
genre_3          3.603794
genre_others     3.389311
reading_years    3.017702
genre_1          2.899258
genre_0          2.570518
books_per_year   2.408520
genre_4          2.229790
genre_2          1.858507
genre_9          1.856211
rating_avg       1.851159
genre_8          1.756641
genre_6          1.679820
genre_7          1.595956
genre_5          1.555201
rating_std       1.196695
